Reconstructing a lineage tree from mutation table

In [1]:
import os
import pandas as pd

In [2]:
name = 'SubC3_train_000X'

In [3]:
project_dir = 'path_to_Subchallenge3/{}'.format(name)

In [4]:
df = pd.read_csv(os.path.join(project_dir,'{}_mutation_table.csv'.format(name)), index_col=0)

In [5]:
symbol_map = {
    0.0: 1,
    '0': 2,
    'A': 3,
    'B':4,
    'C':5,
    'D':6,
    'E':7,
    'F':8,
    'G':9,
    'H':10,
    'I':11,
    'J':12,
    'K':13,
    'L':14,
    'M':15,
    'N':16,
    'O':17,
    'P':18,
    'Q':19,
    'R':20,
    'S':21, # Skipping rare characters as we're running out of standard amino acids
    'T':22,
    'U':23,
    'V':24,
    '-':25, # Inherited deletion is treated as charcter as it carries information (unlike acquisition dropout)
}

In [6]:
for col in df.columns:
    df[col] = df[col].map(symbol_map)

In [7]:
import numpy as np
def np_pars(v1, v2):
    vs = np.array([v1, v2])
    mask = np.isnan(vs[0]) | np.isnan(vs[1])
    filtered = vs[:,np.invert(mask)]
    if not len(filtered[0]):
        ret = 0.5
    else:
        ret = np.sum(np.abs(filtered[1] != filtered[0]))/len(filtered[0])
    return ret

In [8]:
from scipy.spatial.distance import squareform, pdist

data = squareform(pdist(df, metric=np_pars))

In [9]:
dfnn = pd.DataFrame(data).fillna(0.5)

In [10]:
from skbio import DistanceMatrix
from skbio.tree import nj
dm = DistanceMatrix(dfnn, df.index)
tree = nj(dm)
tree.write(os.path.join(project_dir,'NJ.newick'))

/home/dcsoft/miniconda3/envs/clpy36/lib/python3.6/site-packages/skbio/util/_testing.py:15: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as pdt


'/home/dcsoft/s/Ofir/DRAEM/Subchallenge3leaderboard_renamed/SubC3_leaderboard_0003/NJ.newick'

In [11]:
import dendropy
nj = dendropy.Tree.get_from_path(
                    os.path.join(project_dir,'NJ.newick'),
                "newick")

In [12]:
nj.reroot_at_node(nj.find_node_with_taxon_label('root'), update_bipartitions=False)

<Node object at 0x7f85375259e8: 'None' (<Taxon 0x7f8537525a58 'root'>)>

In [13]:
nj.write_to_path(os.path.join(project_dir,'nj.rooted.newick'), schema='newick')